PC로 모바일 인스타그램 데이터 수집 (도전)

In [2]:
import time
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.webdriver.common.appiumby import AppiumBy

capabilities = dict(
    platformName='Android',
    automationName='uiautomator2',
    deviceName='Android',
    appPackage='com.instagram.android',
    appActivity='com.instagram.mainactivity.InstagramMainActivity',
    autoGrantPermissions= True,
    uiautomator2ServerInstallTimeout=60000,  # 60초로 타임아웃 증가
    newCommandTimeout=60000  # 60초
)

appium_server_url = 'http://localhost:4723'

driver = webdriver.Remote(appium_server_url, options=UiAutomator2Options().load_capabilities(capabilities))

time.sleep(5)  # 앱 로딩 대기

try:
    # Appium Inspector에서 복사한 XPATH 사용
    username_field = driver.find_element(AppiumBy.XPATH, "//android.widget.FrameLayout[@resource-id=\"com.instagram.android:id/layout_container_main\"]/android.widget.FrameLayout/android.widget.FrameLayout[2]/android.widget.FrameLayout[1]/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup[1]/android.view.ViewGroup/android.view.ViewGroup/android.widget.EditText")
    password_field = driver.find_element(AppiumBy.XPATH, "//android.widget.FrameLayout[@resource-id=\"com.instagram.android:id/layout_container_main\"]/android.widget.FrameLayout/android.widget.FrameLayout[2]/android.widget.FrameLayout[1]/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup/android.view.ViewGroup[2]/android.view.ViewGroup/android.widget.EditText")
    login_button = driver.find_element(AppiumBy.XPATH, "//android.widget.Button[@content-desc=\"로그인\"]")

    # 아이디
    username_field.send_keys("mlpj_api")
    # 비밀번호
    password_field.send_keys("jy12130!")
    # 로그인 버튼 클릭
    login_button.click()

    time.sleep(10)  # 로그인 처리 대기

    print("로그인 성공")

except Exception as e:
    print(f"로그인 실패: {e}")

로그인 성공


In [3]:
from appium.webdriver.common.appiumby import AppiumBy

# 탐색 탭으로 이동
explore_tab = driver.find_element(AppiumBy.ID, "com.instagram.android:id/search_tab")
explore_tab.click()

In [5]:
# 이유는 모르겠으나 처음 돌리면 실패하고 다시 시도하면 코드 작동이 됨.

# 탐색 필드 클릭 및 검색어 입력
search_field = driver.find_element(AppiumBy.XPATH, "//android.widget.EditText[@resource-id=\"com.instagram.android:id/action_bar_search_edit_text\"]")
# search_field.send_keys('#나이키에어포스1')  # 원하는 검색어
search_field.click()

search_field.send_keys('#팀버랜드6인치')  # 원하는 검색어
time.sleep(3)

# 검색해서 나오는 제일 첫번째 해시태그 클릭
hashtag_click = driver.find_element(AppiumBy.XPATH, "(//android.widget.FrameLayout[@resource-id=\"com.instagram.android:id/row_hashtag_container\"])[1]/android.widget.LinearLayout")
hashtag_click.click()

time.sleep(3)

In [6]:
from appium.webdriver.common.appiumby import AppiumBy

first_post = driver.find_element(AppiumBy.XPATH, "(//android.widget.Button[@content-desc])[1]")
first_post.click()

time.sleep(2)

In [7]:
from appium.webdriver.common.appiumby import AppiumBy
import re
import pandas as pd
from appium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


# 원하는 게시물 수 
max_posts = 2500
# 데이터 담을 변수
results = []
  

def scroll_to_next_post():
    window_size = driver.get_window_size()
    start_y = int(window_size['height'] * 0.7)
    end_y = int(window_size['height'] * 0.3)
    driver.swipe(0, start_y, 0, end_y, 400)
    time.sleep(3)

def save_data_to_csv(data, filename='instagram_post_final.csv'):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")


try:
    post_count = 0
    # seen_posts = set()
    
    while post_count < max_posts:

        try:
            # # 게시물 ID 수집 (예를 들어, 게시물의 사용자 ID를 사용)
            # post_id_ele = WebDriverWait(driver, 10).until(
            #     EC.presence_of_element_located((AppiumBy.ID, "com.instagram.android:id/row_feed_photo_profile_name"))
            # )
            # post_id = post_id_ele.text
            
            # # 중복 확인
            # if post_id in collected_ids:
            #     print(f"중복 게시물 제외: {post_id}")
            #     scroll_to_next_post()
            #     continue
            # else:
            #     collected_ids.add(post_id)


            # 본문
            post_text_ele = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((AppiumBy.XPATH, "//com.instagram.ui.widget.textview.IgTextLayoutView[@resource-id=\"com.instagram.android:id/row_feed_comment_textview_layout\"]"))
            )
            post_text = post_text_ele.text.strip()
            # post_text = driver.find_element(AppiumBy.ID, "com.instagram.android:id/row_feed_comment_textview_layout").text            

            # 본문에서 아이디 제거
            post_texts = ' '.join(post_text.split()[1:])

            # # 중복 검사
            # if post_texts in seen_posts:
            #     print("중복 게시물 발견, 건너뜀")
            #     continue
            # seen_posts.add(post_texts)

            # 더 보기 클릭 후 다시 본문 수집
            try:
                content_click = driver.find_element(AppiumBy.XPATH, "//android.widget.Button[@content-desc='더 보기']")
                content_click.click()
                time.sleep(2)
                
                post_text = driver.find_element(AppiumBy.XPATH, "//com.instagram.ui.widget.textview.IgTextLayoutView[@resource-id=\"com.instagram.android:id/row_feed_comment_textview_layout\"]").text

            except:
                pass
            
            # 본문에 "리그램" 단어가 포함되어 있는지 확인
            if "리그램" in post_texts:
                print(f"리그램이 포함된 게시물 제외: {post_texts}")

                # 스크롤을 시도하여 다음 게시물로 이동
                scroll_to_next_post()
                continue  # 리그램이 포함된 경우 다음 게시물로 넘어감


            # 좋아요 수
            try:
                time.sleep(1)
                post_likes_ele = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//android.widget.TextView[@resource-id=\"com.instagram.android:id/row_feed_textview_likes\"]")))
                # driver.find_element(AppiumBy.XPATH, "//android.widget.TextView[@resource-id=\"com.instagram.android:id/row_feed_textview_likes\"]")
                post_likes_text = post_likes_ele.text

                # 좋아요 수에서 숫자만 추출
                post_likes_match = re.search(r'\d{1,3}(,\d{3})*', post_likes_text)

                post_likes = post_likes_match.group() if post_likes_match else '여러 명'
            
            except Exception as e:
                print(f"좋아요 수 수집 실패: {e}")
                post_likes = "여러 명"


            # 댓글 수 
            try:
                time.sleep(1)
                post_comments_ele = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//android.widget.Button[@resource-id=\"com.instagram.android:id/row_feed_view_all_comments_text\"]")))
                post_comments_text = post_comments_ele.text
                # driver.find_element(AppiumBy.XPATH, "//android.widget.Button[@resource-id=\"com.instagram.android:id/row_feed_view_all_comments_text\"]").text
                
                # 댓글 수에서 숫자만 추출
                post_comments_match = re.search(r'\d{1,3}(,\d{3})*', post_comments_text)

                post_comments = post_comments_match.group() if post_comments_match else '댓글 없음'

            except:
                post_comments = "댓글 없음"


            # 날짜
            time.sleep(1)
            date_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((AppiumBy.XPATH, "//android.widget.TextView[contains(@content-desc, '년') or contains(@content-desc, '월') or contains(@content-desc, '일')]"))
            )
            post_date = date_element.get_attribute("content-desc")
            # date_element = driver.find_element(AppiumBy.XPATH, "//android.widget.TextView[contains(@content-desc, '년')]")
            # post_date = date_element.get_attribute("content-desc")


            # 해시태그
            try:
                time.sleep(1)
                hashtag_elements = driver.find_elements(AppiumBy.XPATH, "//android.widget.Button[contains(@content-desc, '#')]")
                # WebDriverWait(driver, 10).until(
                #     EC.presence_of_all_elements_located((By.XPATH, "//android.widget.Button[contains(@content-desc, '#')]"))
                # )
                # driver.find_elements(AppiumBy.XPATH, "//android.widget.Button[contains(@content-desc, '#')]")
                hashtags = [tag.get_attribute("content-desc") for tag in hashtag_elements]

                if not hashtags:
                    hashtags = [' ']
            
            except:
                hashtags = [' ']


            time.sleep(1)

            results.append({
                "본문": post_texts,
                "좋아요 수": post_likes,
                "댓글 수": post_comments,
                "날짜": post_date,
                "해시태그": hashtags
            })

            post_count += 1  # c게시물 수 증가

        except Exception as e:
            print(f"게시물 수집 실패: {e}")

    
        # 다음 게시물로 스크롤
        scroll_to_next_post()

        time.sleep(2)

except KeyboardInterrupt:
    print("사용자에 의해 중단됨. 데이터 저장 중...")
    save_data_to_csv(results)

except Exception as e:
    print(f"데이터 수집 실패: {e}")

finally:
    save_data_to_csv(results)

# 수집된 결과 출력
for result in results:
    print("본문 : ", result["본문"])
    print("좋아요 수 : ", result["좋아요 수"], "개")
    print("댓글 수 : ", result["댓글 수"], "개")
    print("날짜 : ", result["날짜"])
    print("해시태그 : ", result["해시태그"])
    print("------------------------")

# 데이터프레임 생성
df = pd.DataFrame(results)

# CSV 파일로 저장
df.to_csv('instagram_post_final_1.csv', index=False, encoding='utf-8-sig')

게시물 수집 실패: Message: 
Stacktrace:
NoSuchElementError: An element could not be located on the page using the given search parameters.
    at AndroidUiautomator2Driver.findElOrEls (C:\Users\afds0\.appium\node_modules\appium-uiautomator2-driver\node_modules\appium-android-driver\lib\commands\find.ts:86:11)
    at processTicksAndRejections (node:internal/process/task_queues:95:5)
    at AndroidUiautomator2Driver.findElOrElsWithProcessing (C:\Users\afds0\AppData\Roaming\npm\node_modules\appium\node_modules\@appium\base-driver\lib\basedriver\commands\find.ts:60:12)
    at AndroidUiautomator2Driver.findElement (C:\Users\afds0\AppData\Roaming\npm\node_modules\appium\node_modules\@appium\base-driver\lib\basedriver\commands\find.ts:75:12)
게시물 수집 실패: Message: 
Stacktrace:
NoSuchElementError: An element could not be located on the page using the given search parameters.
    at AndroidUiautomator2Driver.findElOrEls (C:\Users\afds0\.appium\node_modules\appium-uiautomator2-driver\node_modules\appium-a

In [ ]:
# 데이터프레임 출력
print(df)

In [14]:
import pandas as pd

df = pd.DataFrame(results)

print(df)

                                                   본문  좋아요 수   댓글 수  \
0   shoeland_bb 서면제일 나이키 ✔️\n\n📣 나이키 에어포스...    499      3   
1   insight.co.kr_style 버질 아블로가 친구에게 남긴 유작, '올화이트'...    080     19   
2   shoe_bymin 나이키 에어포스1 말라카이트💚🌿\n그린은 어느계절에도 어느 룩에...    926  댓글 없음   
3   insight_trend 올화이트 바디에 사랑스러운 '왕리본' 달아 소장각이라고 난...    114    184   
4   women.streetwear 만능신발 👟🤍나이키 에어포스1 올백​\n​\n#나이키...    877  댓글 없음   
5   insight.co.kr_fashion 루이비통 X 나이키 에어포스 1 콜라보 스니...     30     39   
6   shoe_refur #나이키에어포스1 \n#피스마이너원\n\n안녕하세요 슈리퍼 입니...  2,179      3   
7   menwgirl 색감 존예라는 나이키 #미드나잇네이비\n덩크로우 & 에어포스1 두 ...    189    286   
8   pretty_good_tip 💛존예로운 나이키 에어포스 추천💛\n@@헐 이거 완전 ...    211      1   
9   feed_korea 🧡오렌지포인트🍊 낭낭한\n커플로 신기 좋은 #나이키에어포스1 🧡...    833      8   
10  dispatch_style ⠀\n강렬한 핫핑크 컬러의 나이키 ‘에어 포스 1 - 파...  1,242     24   
11  onl_ootd_ 🍊오렌지포인트🍊 #나이키에어포스1\n스우시 짝짝인 것도 조녜임 진...    298     67   
12  good_tip_gallery 🧡오렌지포인트🍊 낭낭한\n커플로 신기 좋은 #나이키에...    492      4   
13  dt

In [238]:
df.head()

,본문,좋아요 수,날짜,해시태그
0,tip_to.live 보라덕후들은 신어야할 #나이키에어포스 🤩💜\n@@이런 색깔 하...,"1,662",2022년 2월 13일,"[#나이키에어포스, 보라덕후들은 신어야할 #나이키에어포스 🤩💜\n@@이런 색깔 하나..."
1,tip_to.live 보라덕후들은 신어야할 #나이키에어포스 🤩💜\n@@이런 색깔 하...,"1,662",2022년 2월 13일,[#나이키에어포스]
2,tip_to.live 보라덕후들은 신어야할 #나이키에어포스 🤩💜\n@@이런 색깔 하...,438,2022년 2월 13일,"[#남자운동화, #나이키킬샷, #MLB청키라이너, #내셔널지오그래픽운동화, #뉴발란..."
3,menwgirl 샌들의 계절이 얼마 안 남았다~!!\n미리 가을 운동화 준비해보자!...,976,2022년 8월 23일,"[#운동화맛집, #나이키, #MLBKOREA, #내셔널지오그래픽, #뉴발란스]"
4,p__vardo24 (리그램) @befactory3com ☜문의클릭 !\r\n\r\...,여러 명,2023년 8월 9일,"[#발렌시아가, #스피드러너, #모이프, #나이키]"


In [214]:
df.tail()

,본문,좋아요 수,날짜,해시태그
46,ssssom_.m 벌써 연휴 끝이라고..?\n.\n.\n.\n다들 즐추 보내셨나오?...,"3,160",2022년 9월 12일,"[#airforce1, #af1, #sneakerhead, #SNKRS, #snkr..."
47,croket_official 유행 따윈 타지 않는 진리의 ‘에어포스’ 🤍\n흔하지 ...,159,2023년 7월 1일,"[#운동화추천, #나이키추천, #나이키신발, #나이키매니아, #에어포스, #에어포스..."
48,nike_collectors 여러분! 연휴 잘 보내셨나요 🫶🏻 \n연휴 동안에도 많...,"2,115",2023년 10월 4일,[]
49,fashionandstyle.official 김고은의 나이키 슈즈 컬렉션👟💚\n그녀...,334,2023년 10월 2일,"[#나이키데이브레이크, #나이키코르테즈]"
50,krissootal .\n양지네의 N번째의 신발 🥳 유튜브에는 소개했는데 피드에도 ...,127,2023년 10월 2일,"[#나이키에어포스1, #나이키에어포스1폰탄카]"


In [22]:
from appium import webdriver
import time
from selenium.webdriver.common.by import By

# Desired capabilities 설정
desired_caps = {
    "platformName": "Android",
    "appium:automationName": "uiautomator2",
    "deviceName": "R3CT90GE7BK",
    "appPackage": "com.instagram.android",
    "appActivity": "com.instagram.mainactivity.InstagramMainActivity",
    # "autoGrantPermissions": True
}

# 중간에 코드에서 문제가 있을 경우 확인
if desired_caps is None:
    raise ValueError("Desired capabilities cannot be None")

driver = webdriver.Remote("http://localhost:4723", desired_caps)

# Appium 서버에 연결
# driver = webdriver.Remote("http://localhost:4723", desired_caps)
time.sleep(10)  # 앱 로딩 대기

# 로그인 정보 입력
username_field = driver.find_element(By.XPATH, "//input[@name='username']")
password_field = driver.find_element(By.XPATH, "//input[@name='password']")
login_button = driver.find_element(By.XPATH, "//button[@type='submit']")

username_field.send_keys("mlpj_api")
password_field.send_keys("jy12130!")
login_button.click()

time.sleep(10)  # 로그인 처리 대기

AttributeError: 'NoneType' object has no attribute 'to_capabilities'